In [25]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'

In [290]:
from sklearn.naive_bayes import MultinomialNB , GaussianNB
from sklearn.svm import LinearSVC

In [201]:
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [1]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression

import numpy as np



# read train data
with open('./data/train_data.json') as fp:
    json_str = fp.read()
    json_train = json.loads(json_str)
    
with open('./data/test.input.json') as fp:
    json_str = fp.read()
    json_test = json.loads(json_str)
    
with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()
#     json_testout = json.loads(json_str)    

In [16]:
# train data 전처리

train_lst = []
train_row = {}
for i in range(len(json_train)):
    for j, player in enumerate(json_train[i]['participants']): # 그 경기 내에 참여자들
        train_row['champId_'+str(j)] = player['championId'] # 참여자의 champid
        train_row['player_'+str(j)] = player['summonerId'] # 참여자의 smid 
        
    train_row['target'] = json_train[i]['teams'][0]['winner'] # 그 경기에서 100번팀의 승리 여부
    
                  
    train_lst.append(train_row) # 한 경기 참여자들을 모두 lst의 row로
    train_row = {}
    
train_df = pd.DataFrame(train_lst)

In [2]:
# test data 전처리

test_lst = []
test_row = {}
for i in range(len(json_test)):
    for j, player in enumerate(json_test[i]['participants']): # 그 경기 내에 참여자들
        test_row['champId_'+str(j)] = player['championId'] # 참여자의 champid
        test_row['player_'+str(j)] = player['summonerId'] # 참여자의 smid 
        
    #test_row['win_100'] = json_test[i]['teams'][0]['winner'] # 그 경기에서 100번팀의 승리 여부
    
                  
    test_lst.append(test_row) # 한 경기 참여자들을 모두 lst의 row로
    test_row = {}
    
test_df = pd.DataFrame(test_lst)
test_df['target'] = test_out
test_df['target'] = test_df['target'].map(lambda x: True if x == '100' else False)

In [17]:
train_df.columns

Index(['champId_0', 'champId_1', 'champId_2', 'champId_3', 'champId_4',
       'champId_5', 'champId_6', 'champId_7', 'champId_8', 'champId_9',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9', 'target'],
      dtype='object')

In [11]:
test_df.columns

Index(['champId_0', 'champId_1', 'champId_2', 'champId_3', 'champId_4',
       'champId_5', 'champId_6', 'champId_7', 'champId_8', 'champId_9',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9', 'target'],
      dtype='object')

In [18]:
train_df.shape ,test_df.shape

((38679, 21), (2000, 21))

In [14]:
# 모델링

In [33]:
test_df['target']

0        True
1       False
2       False
3       False
4       False
5       False
6        True
7       False
8        True
9       False
10       True
11       True
12       True
13       True
14       True
15       True
16      False
17      False
18      False
19      False
20       True
21      False
22      False
23       True
24       True
25      False
26      False
27      False
28      False
29       True
        ...  
1970     True
1971    False
1972     True
1973     True
1974     True
1975     True
1976     True
1977     True
1978     True
1979     True
1980     True
1981     True
1982     True
1983     True
1984     True
1985    False
1986    False
1987     True
1988     True
1989     True
1990    False
1991    False
1992     True
1993    False
1994     True
1995    False
1996    False
1997     True
1998    False
1999     True
Name: target, Length: 2000, dtype: bool

In [245]:
# submission data 전처리
with open('./data/grading.input.json') as fp:
    json_str = fp.read()
    json_sub = json.loads(json_str)
    
    
sub_lst = []
sub_row = {}
for i in range(len(json_sub)):
    for j, player in enumerate(json_sub[i]['participants']): # 그 경기 내에 참여자들
        test_row['champId_'+str(j)] = player['championId'] # 참여자의 champid
        test_row['player_'+str(j)] = player['summonerId'] # 참여자의 smid 
        
    #test_row['win_100'] = json_test[i]['teams'][0]['winner'] # 그 경기에서 100번팀의 승리 여부
    
                  
    sub_lst.append(sub_row) # 한 경기 참여자들을 모두 lst의 row로
    sub_row = {}
    
sub_df = pd.DataFrame(test_lst)

In [240]:
forest = ExtraTreesClassifier(n_estimators=200).fit(X, y)

In [259]:
# submission 제출 시 
sub = forest.predict(sub_df[col])
sub = pd.Series(sub)
sub = sub.map(lambda x: '100' if x == True else '200')


output_file = 'submission.txt'#sys.argv[2] # 

with open(output_file, 'w') as fp:
    for _test_pred in sub:
        fp.write(_test_pred)
        fp.write('\n')

In [ ]:
# 최고 xgb(0.01, 20)

In [335]:
# test 검증
#forest = ExtraTreesClassifier(n_estimators=1000,random_state=7777).fit(X, y)
forest = xgboost.XGBClassifier(learning_rate=0.01,max_depth=20).fit(X,y)
y_pred = forest.predict(test_df[col])
test_y = test_df['target']
accuracy_score(test_y,y_pred)

0.86699999999999999

In [248]:
accuracy_score(y,y_pred)

1.0

In [337]:
col = ['champId_0', 'champId_1', 'champId_2', 'champId_3', 'champId_4',
       'champId_5', 'champId_6', 'champId_7', 'champId_8', 'champId_9',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9']

X = train_df[col]
y = train_df['target']
from sklearn.model_selection import cross_val_score

cv = KFold(20)

#forest = ExtraTreesClassifier(n_estimators=100, max_depth=300)
forest = xgboost.XGBClassifier(learning_rate=0.01,max_depth=20)

score = cross_val_score(forest, X, y, scoring="accuracy", cv=cv)

print(score)
print(score.mean())
print(score.std())

[ 0.84126163  0.84281282  0.82368149  0.83867632  0.84281282  0.84953464
  0.83453981  0.84281282  0.83298862  0.83712513  0.82833506  0.84332989
  0.83660807  0.84488108  0.83040331  0.84126163  0.83350569  0.84488108
  0.84539814  0.85307812]
0.839396408431
0.00709432807695


In [183]:
col = ['champId_0', 'champId_1', 'champId_2', 'champId_3', 'champId_4',
       'champId_5', 'champId_6', 'champId_7', 'champId_8', 'champId_9',
       'player_0', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5',
       'player_6', 'player_7', 'player_8', 'player_9']

X = train_df[col]
y = train_df['target']
from sklearn.model_selection import cross_val_score

cv = KFold(5)

forest = ExtraTreesClassifier(n_estimators=100, max_depth=300)
#forest = xgboost.XGBClassifier(n_estimators=100, max_depth=2)
from sklearn.linear_model import LogisticRegression

score = cross_val_score(lg, X, y, scoring="accuracy", cv=cv)

print(score)
print(score.mean())
print(score.std())

[ 0.54007239  0.53606515  0.5294726   0.52559462  0.53199741]
0.532640434266
0.00504092779271


In [209]:
X = train_df[col]
y = train_df['target']

cv = KFold(5)

#xgb = xgboost.XGBClassifier(n_estimators=100,learning_rate=0.01)
forest = RandomForestClassifier(n_estimators=200)

#forest = ExtraTreesClassifier(n_estimators=100)
score = cross_val_score(forest, X, y, scoring="accuracy", cv=cv)

print(score)
print(score.mean())
print(score.std())

[ 0.8319545   0.82730093  0.82574974  0.82859359  0.83826761]
0.830373274757
0.00444532369035


In [208]:
X = train_df[col]
y = train_df['target']

cv = KFold(5)

#xgb = xgboost.XGBClassifier(n_estimators=100,learning_rate=0.01)
#forest = RandomForestClassifier(n_estimators=200)
forest = ExtraTreesClassifier(n_estimators=100)
score = cross_val_score(forest, X, y, scoring="accuracy", cv=cv)

print(score)
print(score.mean())
#print(score.std())

[ 0.83298862  0.83143744  0.82755946  0.82872285  0.83516484]
0.831174642317
0.00276991057007


In [141]:
# gridsearch

In [281]:
# 실행 중
from sklearn.model_selection import GridSearchCV
cv = KFold(20)

est_range = [10, 100, 200, 700]


param_grid = [{'n_estimators': est_range}]

gs = GridSearchCV(estimator=ExtraTreesClassifier(), param_grid=param_grid, scoring='accuracy', cv=cv, n_jobs=-1)

%time gs = gs.fit(X, y)

print(gs.best_params_)

CPU times: user 11 s, sys: 776 ms, total: 11.8 s
Wall time: 11min 39s
{'n_estimators': 200}


In [287]:
gs.cv_results_

{'mean_fit_time': array([  1.00397736,  11.25377182,  21.85159556,  73.54405937]),
 'mean_score_time': array([ 0.02120402,  0.20094774,  0.4534251 ,  3.81370357]),
 'mean_test_score': array([ 0.84146436,  0.84570439,  0.84867758,  0.84673854]),
 'mean_train_score': array([ 1.,  1.,  1.,  1.]),
 'param_n_estimators': masked_array(data = [10 100 200 700],
              mask = [False False False False],
        fill_value = ?),
 'params': ({'n_estimators': 10},
  {'n_estimators': 100},
  {'n_estimators': 200},
  {'n_estimators': 700}),
 'rank_test_score': array([4, 3, 1, 2], dtype=int32),
 'split0_test_score': array([ 0.82523268,  0.84126163,  0.82833506,  0.8319545 ]),
 'split0_train_score': array([ 1.,  1.,  1.,  1.]),
 'split10_test_score': array([ 0.82988625,  0.84384695,  0.836091  ,  0.84746639]),
 'split10_train_score': array([ 1.,  1.,  1.,  1.]),
 'split11_test_score': array([ 0.83919338,  0.85625646,  0.85677353,  0.8516029 ]),
 'split11_train_score': array([ 1.,  1.,  1.,  1.])

In [ ]:
# 실행 중
from sklearn.model_selection import GridSearchCV
cv = KFold(20)

#max_depth = [17, 20, 22, 24]
learning_rate = [0.1, 0.08]

param_grid = [{'learning_rate': est_range}]

gs = GridSearchCV(estimator=xgboost.XGBClassifier(max_depth=20), param_grid=param_grid, scoring='accuracy', cv=cv, n_jobs=-1,verbose=1)

%time gs = gs.fit(X, y)

print(gs.best_params_)

Fitting 20 folds for each of 4 candidates, totalling 80 fits
